In [1]:
import tensorflow as tf
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, average_precision_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import model_get
import pickle as pkl
import time
import GPUtil

# Utils

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], False)

def loadGlove(inputpath, outputpath=""):
    data_list = []
    wordEmb = {}
    with open(inputpath) as f:
        for line in f:
            ll = line.strip().split(',')
            ll[0] = str(int(float(ll[0])))
            data_list.append(ll)

            ll_new = [float(i) for i in ll]
            emb = np.array(ll_new[1:], dtype="float32")
            wordEmb[str(int(ll_new[0]))] = emb

    if outputpath != "":
        with open(outputpath) as f:
            for data in data_list:
                f.writelines(' '.join(data))
    return wordEmb

# Setting

In [16]:
import random
import os
seed = 123
random.seed(seed)
os.environ['PYTHONHASHSEED']=str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)


def f1_metric(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    return f1_val


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1_metric', min_delta=0.0001,
    patience=10, verbose=0, mode='max')
callbacks = [early_stopping]

dataset = 'k562'

num_classes = 2
epochs = 30
batch_size = 128
flpath = 'data/'


retrain = False

# Model Training

CrisprIP

In [13]:
print('crispr_ip_model')
encoder_shape = (23, 6)
seg_len, coding_dim = encoder_shape

open_name = 'encoded6x23' + dataset + '.pkl'


print('load data!')
print(open_name)

loaddata = pkl.load(
    open(flpath + open_name, 'rb'),
    encoding='latin1'
)

x_train, x_test, y_train, y_test = train_test_split(
    np.array(loaddata.images),
    loaddata.target,  # loaddata.target,
    stratify=pd.Series(loaddata.target),
    test_size=0.2,
    shuffle=True,
    random_state=42)

x_train, x_val, y_train, y_val = train_test_split(
    x_train,
    y_train,  # loaddata.target,
    stratify=pd.Series(y_train),
    test_size=0.2,
    shuffle=True,
    random_state=42)

neg = 0
for i in y_train:
    if i == 0:
        neg += 1
print(neg)



xtrain, xtest1, ytrain, ytest1, xval, yval, inputshape = model_get.transformIO(
    x_train, x_test, y_train, y_test, x_val, y_val, seg_len, coding_dim, num_classes)

pos_indices = y_train == 1
pos_x, neg_x = xtrain[pos_indices], xtrain[~pos_indices]
pos_y, neg_y = ytrain[pos_indices], ytrain[~pos_indices]
print(len(pos_y))
print(len(neg_y))

pos_ds = tf.data.Dataset.from_tensor_slices((pos_x, pos_y)).repeat()
neg_ds = tf.data.Dataset.from_tensor_slices((neg_x, neg_y)).repeat()

resampled_ds = tf.data.experimental.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5], seed=seed)
resampled_ds = resampled_ds.batch(batch_size).prefetch(2)
resampled_steps_per_epoch = np.ceil(2 * neg / batch_size)
print(resampled_steps_per_epoch)

test_ds = tf.data.Dataset.from_tensor_slices((xval, yval)).cache()
test_ds = test_ds.batch(batch_size)


print('Training!!')
# Record the start time 
start_time = time.time()
# Fetch GPU details and record initial memory usage
GPUs = GPUtil.getGPUs()
gpu = GPUs[0]
initial_memory = gpu.memoryUsed
print(f"Initial GPU Memory Usage: {initial_memory} MB")

crispr_ip_model = model_get.crispr_ip(test_ds, resampled_steps_per_epoch, resampled_ds, xtrain, ytrain,
                                       xtest1,
                                       ytest1,
                                       inputshape, num_classes, batch_size, epochs, callbacks,
                                       dataset, retrain)

# Record the end time 
end_time = time.time()
# Calculate the time 
time_taken = end_time - start_time
print(f"Training Time: {time_taken:.2f} seconds")
# After training, record the final memory usage and calculate the difference
final_memory = gpu.memoryUsed
print(f"Final GPU Memory Usage: {final_memory} MB")


yscore = crispr_ip_model.predict(xtest1)
ypred = np.argmax(yscore, axis=1)
yscore = yscore[:, 1]
ytest = np.argmax(ytest1, axis=1)
eval_funs = [accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, average_precision_score]
eval_fun_names = ['Accuracy', 'F1 score', 'Precision', 'Recall', 'ROC AUC', 'PR AUC']
eval_fun_types = [True, True, True, True, False, False]
for index_f, function in enumerate(eval_funs):
    if eval_fun_types[index_f]:
        score = np.round(function(ytest, ypred), 4)
    else:
        score = np.round(function(ytest, yscore), 4)
    print('{:<15}{:>15}'.format(eval_fun_names[index_f], score))

crispr_ip_model
load data!
encoded6x23ELEVATION.pkl
136884
xtrain shape: (136916, 1, 23, 6)
136916 train samples
42787 test samples
34230 val samples
32
136884
2139.0
Training!!
Initial GPU Memory Usage: 6838.0 MB
Epoch 1/30
2139/2139 [==============================] - 21s 10ms/step - loss: 0.6964 - accuracy: 0.4998 - f1_metric: 0.4998 - val_loss: 0.7098 - val_accuracy: 0.0025 - val_f1_metric: 0.0025
Epoch 2/30
2139/2139 [==============================] - 20s 10ms/step - loss: 0.6935 - accuracy: 0.5005 - f1_metric: 0.5005 - val_loss: 0.7123 - val_accuracy: 2.3371e-04 - val_f1_metric: 2.3321e-04
Epoch 3/30
2139/2139 [==============================] - 20s 10ms/step - loss: 0.6932 - accuracy: 0.5004 - f1_metric: 0.5004 - val_loss: 0.6944 - val_accuracy: 0.0028 - val_f1_metric: 0.0028
Epoch 4/30
2139/2139 [==============================] - 21s 10ms/step - loss: 0.6932 - accuracy: 0.4999 - f1_metric: 0.4999 - val_loss: 0.6787 - val_accuracy: 0.9997 - val_f1_metric: 0.9997
Epoch 5/30
2139/21

cnn_std

In [17]:
print('cnn_std')
encoder_shape = (23, 4)
seg_len, coding_dim = encoder_shape
open_name = 'encoded4x23' + dataset + '.pkl'

print('load data!')
print(open_name)

loaddata = pkl.load(
    open(flpath + open_name, 'rb'),
    encoding='latin1'
)

x_train, x_test, y_train, y_test = train_test_split(
    np.array(loaddata.images),
    loaddata.target,
    stratify=pd.Series(loaddata.target),
    test_size=0.2,
    shuffle=True,
    random_state=42)

x_train, x_val, y_train, y_val = train_test_split(
    x_train,
    y_train,
    stratify=pd.Series(y_train),
    test_size=0.2,
    shuffle=True,
    random_state=42)

neg = 0
for i in y_train:
    if i == 0:
        neg += 1
print(neg)

xtrain, xtest2, ytrain, ytest2, xval, yval, input_shape = model_get.cnn_std_transformIO(x_train, x_test,
                                                                                         y_train,
                                                                                         y_test, x_val, y_val,
                                                                                         seg_len, coding_dim,
                                                                                         num_classes)
pos_indices = y_train == 1
pos_x, neg_x = xtrain[pos_indices], xtrain[~pos_indices]
pos_y, neg_y = ytrain[pos_indices], ytrain[~pos_indices]
print(len(pos_y))
print(len(neg_y))

pos_ds = tf.data.Dataset.from_tensor_slices((pos_x, pos_y)).repeat()
neg_ds = tf.data.Dataset.from_tensor_slices((neg_x, neg_y)).repeat()

resampled_ds = tf.data.experimental.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5], seed=seed)
resampled_ds = resampled_ds.batch(batch_size).prefetch(2)
resampled_steps_per_epoch = np.ceil(2 * neg / batch_size)
print(resampled_steps_per_epoch)

test_ds = tf.data.Dataset.from_tensor_slices((xval, yval)).cache()
test_ds = test_ds.batch(batch_size)

print('Training!!')
# Record the start time 
start_time = time.time()
# Fetch GPU details and record initial memory usage
GPUs = GPUtil.getGPUs()
gpu = GPUs[0]
initial_memory = gpu.memoryUsed
print(f"Initial GPU Memory Usage: {initial_memory} MB")
cnn_std_model = model_get.cnn_std(test_ds, resampled_steps_per_epoch, resampled_ds, xtrain, ytrain,
                                   xtest2,
                                   ytest2, num_classes, batch_size, epochs, callbacks,
                                   dataset, retrain)

# Record the end time 
end_time = time.time()
# Calculate the time 
time_taken = end_time - start_time
print(f"Training Time: {time_taken:.2f} seconds")
# After training, record the final memory usage and calculate the difference
final_memory = gpu.memoryUsed
print(f"Final GPU Memory Usage: {final_memory} MB")

yscore = cnn_std_model.predict(xtest2)
ypred = np.argmax(yscore, axis=1)
yscore = yscore[:, 1]
ytest = np.argmax(ytest2, axis=1)
eval_funs = [accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, average_precision_score]
eval_fun_names = ['Accuracy', 'F1 score', 'Precision', 'Recall', 'ROC AUC', 'PR AUC']
eval_fun_types = [True, True, True, True, False, False]
for index_f, function in enumerate(eval_funs):
    if eval_fun_types[index_f]:
        score = np.round(function(ytest, ypred), 4)
    else:
        score = np.round(function(ytest, yscore), 4)
    print('{:<15}{:>15}'.format(eval_fun_names[index_f], score))

cnn_std
load data!
encoded4x23k562.pkl
11722
xtrain shape: (11797, 1, 23, 4)
11797 train samples
3687 test samples
2950 xval samples
75
11722
184.0
Training!!
Initial GPU Memory Usage: 2144.0 MB
Epoch 1/30
184/184 [==============================] - 1s 5ms/step - loss: 0.6932 - accuracy: 0.4976 - f1_metric: 0.4949 - val_loss: 0.6930 - val_accuracy: 0.9936 - val_f1_metric: 0.9938
Epoch 2/30
184/184 [==============================] - 1s 4ms/step - loss: 0.6932 - accuracy: 0.4982 - f1_metric: 0.4982 - val_loss: 0.6918 - val_accuracy: 0.9936 - val_f1_metric: 0.9938
Epoch 3/30
184/184 [==============================] - 1s 4ms/step - loss: 0.6932 - accuracy: 0.4972 - f1_metric: 0.4972 - val_loss: 0.6932 - val_accuracy: 0.0064 - val_f1_metric: 0.0062
Epoch 4/30
184/184 [==============================] - 1s 4ms/step - loss: 0.6932 - accuracy: 0.5032 - f1_metric: 0.5032 - val_loss: 0.6886 - val_accuracy: 0.9936 - val_f1_metric: 0.9938
Epoch 5/30
184/184 [==============================] - 1s 4ms/

crisprDNT

In [11]:
print('crisprDNT')
open_name = 'encodedmismatchtype14x23' + dataset + '.pkl'
encoder_shape = (23, 14)
seg_len, coding_dim = encoder_shape

print('load data!')
print(open_name)

loaddata = pkl.load(
    open(flpath + open_name, 'rb'),
    encoding='latin1'
)

x_train, x_test, y_train, y_test = train_test_split(
    loaddata.images,
    loaddata.target,
    stratify=pd.Series(loaddata.target),
    test_size=0.2,
    shuffle=True,
    random_state=42)

x_train, x_val, y_train, y_val = train_test_split(
    x_train,
    y_train,
    stratify=pd.Series(y_train),
    test_size=0.2,
    shuffle=True,
    random_state=42)

neg = 0
for i in y_train:
    if i == 0:
        neg += 1
print(neg)

xtrain, xtest3, ytrain, ytest3, xval, yval, inputshape = model_get.transformIO(
    x_train, x_test, y_train, y_test, x_val, y_val, seg_len, coding_dim, num_classes)

pos_indices = y_train == 1
pos_x, neg_x = xtrain[pos_indices], xtrain[~pos_indices]
pos_y, neg_y = ytrain[pos_indices], ytrain[~pos_indices]
print(len(pos_y))
print(len(neg_y))
pos_ds = tf.data.Dataset.from_tensor_slices((pos_x, pos_y)).repeat()
neg_ds = tf.data.Dataset.from_tensor_slices((neg_x, neg_y)).repeat()

resampled_ds = tf.data.experimental.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5], seed=seed)
resampled_ds = resampled_ds.batch(batch_size).prefetch(2)
resampled_steps_per_epoch = np.ceil(2 * neg / batch_size)
print(resampled_steps_per_epoch)

test_ds = tf.data.Dataset.from_tensor_slices((xval, yval)).cache()
test_ds = test_ds.batch(batch_size)

print('Training!!')
# Record the start time 
start_time = time.time()
# Fetch GPU details and record initial memory usage
GPUs = GPUtil.getGPUs()
gpu = GPUs[0]
initial_memory = gpu.memoryUsed
print(f"Initial GPU Memory Usage: {initial_memory} MB")

crisprDNT = model_get.crisprDNT(test_ds, resampled_steps_per_epoch, resampled_ds, xtrain, ytrain,
                                     xtest3,
                                     ytest3,
                                     inputshape, num_classes, batch_size, epochs, callbacks,
                                     dataset, retrain)

# Record the end time 
end_time = time.time()
# Calculate the time 
time_taken = end_time - start_time
print(f"Training Time: {time_taken:.2f} seconds")
# After training, record the final memory usage and calculate the difference
final_memory = gpu.memoryUsed
print(f"Final GPU Memory Usage: {final_memory} MB")


yscore = crisprDNT.predict(xtest3)
ypred = np.argmax(yscore, axis=1)
yscore = yscore[:, 1]
ytest = np.argmax(ytest3, axis=1)
eval_funs = [accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, average_precision_score]
eval_fun_names = ['Accuracy', 'F1 score', 'Precision', 'Recall', 'ROC AUC', 'PR AUC']
eval_fun_types = [True, True, True, True, False, False]
for index_f, function in enumerate(eval_funs):
    if eval_fun_types[index_f]:
        score = np.round(function(ytest, ypred), 4)
    else:
        score = np.round(function(ytest, yscore), 4)
    print('{:<15}{:>15}'.format(eval_fun_names[index_f], score))

crisprDNT
load data!
encodedmismatchtype14x23ELEVATION.pkl
136884
xtrain shape: (136916, 1, 23, 14)
136916 train samples
42787 test samples
34230 val samples
32
136884
2139.0
Training!!
Initial GPU Memory Usage: 7964.0 MB
Epoch 1/30
2139/2139 [==============================] - 546s 255ms/step - loss: 0.0618 - accuracy: 0.9732 - f1_metric: 0.9732 - val_loss: 0.1244 - val_accuracy: 0.9467 - val_f1_metric: 0.9467
Epoch 2/30
2139/2139 [==============================] - 538s 252ms/step - loss: 0.0036 - accuracy: 0.9991 - f1_metric: 0.9991 - val_loss: 0.0090 - val_accuracy: 0.9972 - val_f1_metric: 0.9972
Epoch 3/30
2139/2139 [==============================] - 531s 248ms/step - loss: 0.0023 - accuracy: 0.9995 - f1_metric: 0.9995 - val_loss: 0.0060 - val_accuracy: 0.9991 - val_f1_metric: 0.9991
Epoch 4/30
2139/2139 [==============================] - 527s 246ms/step - loss: 0.0020 - accuracy: 0.9996 - f1_metric: 0.9996 - val_loss: 0.0060 - val_accuracy: 0.9994 - val_f1_metric: 0.9994
Epoch 5/30

crisprNet

In [12]:
print('CRISPR_Net')
open_name = 'encoded7x23' + dataset + '.pkl'
encoder_shape = (23, 7)
seg_len, coding_dim = encoder_shape
print('load data!')
print(open_name)
loaddata = pkl.load(
    open(flpath + open_name, 'rb'),
    encoding='latin1'
)

x_train, x_test, y_train, y_test = train_test_split(
    loaddata.images,
    loaddata.target,
    stratify=pd.Series(loaddata.target),
    test_size=0.2,
    shuffle=True,
    random_state=42)

x_train, x_val, y_train, y_val = train_test_split(
    x_train,
    y_train,
    stratify=pd.Series(y_train),
    test_size=0.2,
    shuffle=True,
    random_state=42)

neg = 0
for i in y_train:
    if i == 0:
        neg += 1
print(neg)

xtrain, xtest4, ytrain, ytest4, xval, yval, inputshape = model_get.CRISPR_Net_transformIO(
    x_train, x_test, y_train, y_test, x_val, y_val, seg_len, coding_dim, num_classes)

pos_indices = y_train == 1
pos_x, neg_x = xtrain[pos_indices], xtrain[~pos_indices]
pos_y, neg_y = ytrain[pos_indices], ytrain[~pos_indices]
print(len(pos_y))
print(len(neg_y))

pos_ds = tf.data.Dataset.from_tensor_slices((pos_x, pos_y)).repeat()
neg_ds = tf.data.Dataset.from_tensor_slices((neg_x, neg_y)).repeat()
resampled_ds = tf.data.experimental.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5], seed=seed)
resampled_ds = resampled_ds.batch(batch_size).prefetch(2)

resampled_steps_per_epoch = np.ceil(2 * neg / batch_size)
print(resampled_steps_per_epoch)

test_ds = tf.data.Dataset.from_tensor_slices((xval, yval)).cache()
test_ds = test_ds.batch(batch_size)

print('Training!!')
# Record the start time 
start_time = time.time()
# Fetch GPU details and record initial memory usage
GPUs = GPUtil.getGPUs()
gpu = GPUs[0]
initial_memory = gpu.memoryUsed
print(f"Initial GPU Memory Usage: {initial_memory} MB")

CRISPR_Net_model = model_get.CRISPR_Net_model(test_ds, resampled_steps_per_epoch, resampled_ds,
                                               xtrain, ytrain,
                                               xtest4,
                                               ytest4,
                                               inputshape, num_classes, batch_size, epochs, callbacks,
                                               dataset, retrain)

# Record the end time 
end_time = time.time()
# Calculate the time 
time_taken = end_time - start_time
print(f"Training Time: {time_taken:.2f} seconds")
# After training, record the final memory usage and calculate the difference
final_memory = gpu.memoryUsed
print(f"Final GPU Memory Usage: {final_memory} MB")


yscore = CRISPR_Net_model.predict(xtest4)
ypred = np.argmax(yscore, axis=1)
yscore = yscore[:, 1]
ytest = np.argmax(ytest4, axis=1)
eval_funs = [accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, average_precision_score]
eval_fun_names = ['Accuracy', 'F1 score', 'Precision', 'Recall', 'ROC AUC', 'PR AUC']
eval_fun_types = [True, True, True, True, False, False]
for index_f, function in enumerate(eval_funs):
    if eval_fun_types[index_f]:
        score = np.round(function(ytest, ypred), 4)
    else:
        score = np.round(function(ytest, yscore), 4)
    print('{:<15}{:>15}'.format(eval_fun_names[index_f], score))

CRISPR_Net
load data!
encoded7x23ELEVATION.pkl
136884
xtrain shape: (136916, 1, 23, 7)
136916 train samples
42787 test samples
34230 val samples
32
136884
2139.0
Training!!
Initial GPU Memory Usage: 6838.0 MB
Epoch 1/30
2139/2139 [==============================] - 20s 9ms/step - loss: 0.0535 - accuracy: 0.9802 - f1_metric: 0.9802 - val_loss: 0.0141 - val_accuracy: 0.9954 - val_f1_metric: 0.9952
Epoch 2/30
2139/2139 [==============================] - 19s 9ms/step - loss: 0.0094 - accuracy: 0.9970 - f1_metric: 0.9970 - val_loss: 0.0058 - val_accuracy: 0.9985 - val_f1_metric: 0.9985
Epoch 3/30
2139/2139 [==============================] - 19s 9ms/step - loss: 0.0073 - accuracy: 0.9974 - f1_metric: 0.9974 - val_loss: 0.0034 - val_accuracy: 0.9993 - val_f1_metric: 0.9993
Epoch 4/30
2139/2139 [==============================] - 19s 9ms/step - loss: 0.0036 - accuracy: 0.9989 - f1_metric: 0.9989 - val_loss: 0.0053 - val_accuracy: 0.9993 - val_f1_metric: 0.9991
Epoch 5/30
2139/2139 [=============

cnnCRISPR

In [39]:
print('cnn_crispr model')
print("GloVe model loaded")
VOCAB_SIZE = 16  # 4**3
EMBED_SIZE = 100
glove_inputpath = "data/keras_GloVeVec_" + dataset + "_5_100_10000.csv"
# load GloVe model
model_glove = loadGlove(glove_inputpath)
embedding_weights = np.zeros((VOCAB_SIZE, EMBED_SIZE))
for i in range(VOCAB_SIZE):
    embedding_weights[i, :] = model_glove[str(i)]

open_name = 'encoded_CnnCrispr_' + dataset + '.pkl'


print('load data!')
print(open_name)

loaddata = pkl.load(
    open(flpath + open_name, 'rb'),
    encoding='latin1'
)

x_train, x_test, y_train, y_test = train_test_split(
    np.array(loaddata.images),
    loaddata.target,
    stratify=pd.Series(loaddata.target),
    test_size=0.2,
    shuffle=True,
    random_state=42)

x_train, x_val, y_train, y_val = train_test_split(
    x_train,
    y_train,
    stratify=pd.Series(y_train),
    test_size=0.2,
    shuffle=True,
    random_state=42)

neg = 0
for i in y_train:
    if i == 0:
        neg += 1
print(neg)

xtrain, xtest5, ytrain, ytest5, xval, yval = model_get.offt_transformIO(x_train, x_test, y_train, y_test,
                                                                         x_val, y_val, num_classes)

pos_indices = y_train == 1
pos_x, neg_x = xtrain[pos_indices], xtrain[~pos_indices]
pos_y, neg_y = ytrain[pos_indices], ytrain[~pos_indices]
print(len(pos_y))
print(len(neg_y))

pos_ds = tf.data.Dataset.from_tensor_slices((pos_x, pos_y)).repeat()
neg_ds = tf.data.Dataset.from_tensor_slices((neg_x, neg_y)).repeat()

resampled_ds = tf.data.experimental.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5], seed=seed)
resampled_ds = resampled_ds.batch(batch_size).prefetch(2)
resampled_steps_per_epoch = np.ceil(2 * neg / batch_size)
print(resampled_steps_per_epoch)

test_ds = tf.data.Dataset.from_tensor_slices((xval, yval)).cache()
test_ds = test_ds.batch(batch_size)

print('Training!!')

# Record the start time 
start_time = time.time()
# Fetch GPU details and record initial memory usage
GPUs = GPUtil.getGPUs()
gpu = GPUs[0]
initial_memory = gpu.memoryUsed
print(f"Initial GPU Memory Usage: {initial_memory} MB")

CnnCrispr_model = model_get.CnnCrispr(embedding_weights, test_ds, resampled_steps_per_epoch, resampled_ds,
                                       xtrain, ytrain,
                                       xtest5,
                                       ytest5, num_classes, batch_size, epochs, callbacks,
                                       dataset, retrain)

# Record the end time 
end_time = time.time()
# Calculate the time 
time_taken = end_time - start_time
print(f"Training Time: {time_taken:.2f} seconds")
# After training, record the final memory usage and calculate the difference
final_memory = gpu.memoryUsed
print(f"Final GPU Memory Usage: {final_memory} MB")


yscore = CnnCrispr_model.predict(xtest5)
ypred = np.argmax(yscore, axis=1)
yscore = yscore[:, 1]
ytest = np.argmax(ytest5, axis=1)
eval_funs = [accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, average_precision_score]
eval_fun_names = ['Accuracy', 'F1 score', 'Precision', 'Recall', 'ROC AUC', 'PR AUC']
eval_fun_types = [True, True, True, True, False, False]
for index_f, function in enumerate(eval_funs):
    if eval_fun_types[index_f]:
        score = np.round(function(ytest, ypred), 4)
    else:
        score = np.round(function(ytest, yscore), 4)
    print('{:<15}{:>15}'.format(eval_fun_names[index_f], score))

cnn_crispr model
GloVe model loaded
load data!
encoded_CnnCrispr_SITE.pkl
136937
xtrain shape: (139348, 23)
139348 train samples
43547 test samples
34838 val samples
2411
136937
2140.0
Training!!
Initial GPU Memory Usage: 7234.0 MB
Epoch 1/30
2140/2140 [==============================] - 32s 15ms/step - loss: 0.7071 - accuracy: 0.4996 - f1_metric: 0.4996 - val_loss: 0.6972 - val_accuracy: 0.4422 - val_f1_metric: 0.4413
Epoch 2/30
2140/2140 [==============================] - 31s 14ms/step - loss: 0.6934 - accuracy: 0.5006 - f1_metric: 0.5006 - val_loss: 0.7234 - val_accuracy: 0.1222 - val_f1_metric: 0.1220
Epoch 3/30
2140/2140 [==============================] - 31s 15ms/step - loss: 0.6933 - accuracy: 0.5004 - f1_metric: 0.5004 - val_loss: 0.6571 - val_accuracy: 0.2033 - val_f1_metric: 0.2052
Epoch 4/30
2140/2140 [==============================] - 30s 14ms/step - loss: 0.6932 - accuracy: 0.5003 - f1_metric: 0.5003 - val_loss: 45637681152.0000 - val_accuracy: 0.7902 - val_f1_metric: 0.790